In [260]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime,timezone

from configupdater import ConfigUpdater
# pip install ConfigUpdater

from dotenv import dotenv_values

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account

In [261]:
#dt_imported=datetime.now(timezone.utc) # utc
dt_imported=datetime.now()
dt_imported=datetime.strptime(dt_imported.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
print(dt_imported)

2023-12-15 17:10:27


In [262]:
view_name="pmr_project"
listIDs=[25]
key_id="project_id"


In [263]:


projectId='kku-intern-dataai'  # smart-data-ml
dataset_id='Demo_PMReport'
table_name=view_name.replace("pmr_","")
table_id = f"{projectId}.{dataset_id}.{table_name}"
print(table_id)
to_bq_mode="WRITE_APPEND"

kku-intern-dataai.Demo_PMReport.project


# Read Configuration File and Initialize BQ Object

In [264]:
updater = ConfigUpdater()
updater.read("view_updater.cfg")

env_path='.env'
config = dotenv_values(dotenv_path=env_path)

In [265]:
credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\kku-intern-dataai-a5449aee8483.json')
client = bigquery.Client(credentials= credentials,project=projectId)
print(client)

In [266]:
last_imported=datetime.strptime(updater["metadata"][view_name].value,"%Y-%m-%d %H:%M:%S")
print(last_imported)

2023-12-15 17:06:16


# Postgres &BigQuery

In [267]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [268]:
def get_bq_table():
 try:
    table=client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
    print(table.schema)
    return True
 except NotFound:
    raise Exception("Table {} is not found.".format(table_id))
    
def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition=to_bq_mode,
        )

        job = client.load_table_from_dataframe(
            df_trasns, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_trasns), f"Imported data to {table_id} on bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 



# Load view and transform

In [269]:
if len(listIDs)>1:
 sql_view=f"select *  from {view_name}  where {key_id} in {tuple(listIDs)}"
else:
 sql_view=f"select *  from {view_name}  where {key_id} ={listIDs[0]}"
print(sql_view)
df=list_data(sql_view,None,get_postgres_conn())
df['imported_at']=dt_imported
df

select *  from pmr_project  where project_id =25


,project_id,enq,project_name,project_start,project_end,company_id,company,has_pm,imported_at
0,25,ENQ/66-141223#3,NetApp#2-KKU,2023-12-01,2024-02-29,150,KKU,False,2023-12-15 17:10:27


# Ingest into BQ

In [270]:
if get_bq_table():
    try:
        insertDataFrameToBQ(df)
    except Exception as ex:
        raise ex

Table kku-intern-dataai.Demo_PMReport.project already exists.
[SchemaField('project_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('enq', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('project_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('project_start', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('project_end', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('company_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('company', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('has_pm', 'BOOLEAN', 'NULLABLE', None, None, (), None), SchemaField('imported_at', 'DATETIME', 'REQUIRED', None, None, (), None)]
Total  1 Imported data to kku-intern-dataai.Demo_PMReport.project on bigquery successfully


In [271]:
# Update Last Update

In [272]:
updater["metadata"][view_name].value=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
updater.update_file() 

<ConfigUpdater [
    <Section: 'metadata' [
        <Option: pmr_project = '2023-12-15 17:10:27'>
        <Option: pmr_pm_plan = '2000-01-01 00:05:00'>
    ]>
]>